In [1]:
import pickle
import yaml
import pandas as pd
import pickle
import sys
import numpy as np

import matplotlib.pyplot as plot

In [2]:
def loadConfig():    
    with open('./light_config_hulk.yaml', "r") as ymlfile: #hulk, slowloris; GE, heartbleed, slowhttp
        cfg = yaml.load(ymlfile)
    return cfg

config = loadConfig()

/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


## Loading CIC Wed dataset

In [3]:
config['metadata']['artefact']
# config['metadata']['artefact'] = 'artefact_seqlen200hash750'

'artefact_hulk'

##### Training set

In [4]:
# N1 data
df_normal = pd.read_csv(config['metadata']['uniqueID'] + '/' + config['metadata']['artefact'] + '/' + 'N1.csv')
print("size of normal data: ", len(df_normal))
# df_normal.head()

size of normal data:  31374


In [5]:
# Entire train atk set. 
df_attackfull = pd.read_csv(config['metadata']['uniqueID'] + '/' + config['metadata']['artefact'] + '/' + 'A1_full.csv')
print("size of full attack data: ", len(df_attackfull))

size of full attack data:  21987


In [6]:
# Join and shuffle train attack and normal data
# df_train = df_normal.append(df_attackfull).sample(frac=1, replace=False, random_state=39).reset_index(drop=True)

df_train = df_attackfull
print("size of df_train: ", len(df_train))

size of df_train:  21987


In [7]:
# Add labels to train set
attacker='172.16.0.1'; victim='192.168.10.50'
df_train['Attack'] = 0

# if IP == attacker or IP == victim
df_train.loc[df_train['Source IP']==attacker, 'Attack'] = 1
df_train.loc[df_train['Source IP']==victim, 'Attack'] = 1

In [8]:
(df_train['Attack']).value_counts()

1    16704
0     5283
Name: Attack, dtype: int64

In [9]:
num_atk = df_train['Attack'].values.sum()  # True
num_atk

16704

In [10]:
num_norm = len(df_train) - df_train['Attack'].sum()  
num_norm

5283

In [11]:
# count atk percentage in test set
# num_norm, num_atk = (df_test['Attack']).value_counts()

print("Length of test data: ", len(df_train))
print("Number of normal traffic: ", num_norm)
print("Number of true attacks: ", num_atk)
print("Percent attacks: ", num_atk / len(df_train))

Length of test data:  21987
Number of normal traffic:  5283
Number of true attacks:  16704
Percent attacks:  0.7597216537044618


##### Test set

In [12]:
# Load test set
df_test = pd.read_csv(config['metadata']['uniqueID'] + '/' + config['metadata']['artefact'] + '/' + 'TEST.csv')
print("size of df_test: ", len(df_test))

size of df_test:  11710


In [13]:
# Add labels to test set
attacker='172.16.0.1'; victim='192.168.10.50'
df_test['Attack'] = 0

# if IP == attacker or IP == victim
df_test.loc[df_test['Source IP']==attacker, 'Attack'] = 1
df_test.loc[df_test['Source IP']==victim, 'Attack'] = 1

df_test = df_test.sample(frac=1, random_state=39, replace=False)

In [14]:
(df_test['Attack']).value_counts()

1    7158
0    4552
Name: Attack, dtype: int64

In [15]:
num_atk = df_test['Attack'].values.sum()  # True
num_atk

7158

In [16]:
num_norm = len(df_test) - df_test['Attack'].sum()  
num_norm

4552

In [17]:
# count atk percentage in test set
# num_norm, num_atk = (df_test['Attack']).value_counts()

print("Length of test data: ", len(df_test))
print("Number of normal traffic: ", num_norm)
print("Number of true attacks: ", num_atk)
print("Percent attacks: ", num_atk / len(df_test))

Length of test data:  11710
Number of normal traffic:  4552
Number of true attacks:  7158
Percent attacks:  0.611272416737831
